# 관련 링크
-kkanggle의 [Lyft Motion Prediction for Autonomous Vehicles 경진대회 데이터 셋](https://www.kaggle.com/competitions/lyft-motion-prediction-autonomous-vehicles/data)을 사용함
- 2025-02-23 현재시점 기준 2년전 마지막 업데이트 된 프로젝트 [L5kit모듈](https://github.com/woven-planet/l5kit/tree/master)과 smaple notebook을 참조하여 데이터를 로드 및 학습에 사용
- [L5kit 사용 설명 링크](https://woven-planet.github.io/l5kit/)
- lyft 모듈은 [kaggle_L5kit](https://www.kaggle.com/code/philculliton/kaggle-l5kit)이라는 유틸리티 스크립트를 제공.(커널에 첨부 하면 최신 버전의 모든 종속성 가능하다함.)
- train.zarr과 test.zarr만 가지고 할 수 있다곤 함.

In [ ]:
# 데이터 file 구조
'''
prediction-dataset/
  ├── scenes/
  │   ├── train.zarr
  │   ├── validate.zarr
  │   ├── test.zarr
  │   └── train_full.zarr  # 전체 
  ├── aerial_map/
  │   └── aerial_map.png
  │   └── lyft.tfw lyft.tlf
  ├── semantic_map/
  │   └── semantic_map.pb
  └── meta.json
'''

In [ ]:
# 필요 패키지
# Python 3(3.6 이상) 이라고 하는데 3.12 에서는 충돌 나는 것으로 보임. 아래는 3.7 ver에서 했음.
'''
pip install l5kit
pip install torch torchvision
pip install pytorch-lightning
pip install zarr
pip install numpy pandas matplotlib seaborn
pip install opencv-python
'''

SyntaxError: invalid syntax (3480292577.py, line 3)

### 다운 받은 파일 확장자 관련 설명
- csv(comma-separated values) : 샘플데이터 2개 있음   
- meta.json : 변환 행렬 -> 세계 좌표계에서 ECEF좌표계로 변환 하는 행렬,ECEF 좌표 계에서 항공 사진 좌표계로 변환 하는 행렬     
- .pb(protocol buffer) : 데이터를 serialize하는 방식 중 하나인데 간단히 binary로 바꿔서 통신에 유리하도록 하는 방법 벡터 기반의 도로 정보(정지선 신호등 차선 교차로)의 데이터가 있음   
- png : 위성 항공사진으로 도로와 건물 의 실제 이미지 제공   
- tfw, tlf : geoTIFF 파일의 격자 정보를 지정한 외부 텍스트 파일과 tiff파일(이미지 혹은 래스터 그래픽 정보저장 파일일)
- npz : 테스트 세트에 대한 부울 정보 마스크에 포함된 모든 에이전트만 제출 해야한다고함. -> np를 저장하기 위한 압출 파일이라고 생각하면된다고함.   
- zarr : 맷의 데이터는 N차원 배열을 효율적으로 저장하고 접근할 수 있도록 설계되어 있음. 각 주행 에피소드는 여러 프레임으로 구성되며, 각 프레임은 특정 시점의 차량 상태와 주변 객체 정보를 포함함.
  - 프레임 : 자율주행 차량의 위치 방향 속도 등의 정보
  - 에이전트 : 주변의 다른 차량 자전거 보행자 등의 위치 속도 방향등의 정보
  - 트래픽 라이트 :  신호등의 상태정보

In [2]:
from l5kit.data import ChunkedDataset

# 데이터 경로 설정 dataset_path = r""\validate.zarr"
dataset_path = r"lyft-motion-prediction-autonomous-vehicles/scenes/validate.zarr"

# 데이터 불러오기
zarr_dataset = ChunkedDataset(dataset_path).open()
print("Dataset Loaded!")

# 첫 번째 씬 확인
first_scene = zarr_dataset.scenes[0]
print("First Scene:", first_scene)

# 첫 번째 프레임 확인
first_frame = zarr_dataset.frames[0]
print("First Frame:", first_frame)

# 첫 번째 에이전트 확인
first_agent = zarr_dataset.agents[0]
print("First Agent:", first_agent)

# 첫 번째 트래픽 라이트 얼굴 정보 가져오기
first_traffic_light_face = zarr_dataset.tl_faces[0]
print("First Traffic Light Face:", first_traffic_light_face)


Dataset Loaded!
First Scene: ([  0, 248], 'host-a116', 1576254970817669888, 1576254995817669888)
First Frame: (1576254971002364806, [ 0, 56], [0, 0], [  541.62249756, -2402.70581055,   288.62399292], [[-0.42925233, -0.90311611,  0.01112122], [ 0.90317595, -0.42916176,  0.00966374], [-0.00395468,  0.0141926 ,  0.99989146]])
First Agent: ([  545.47155762, -2411.68945312], [4.427541 , 1.8168873, 1.6035007], 2.0373902, [0., 0.], 1, [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
First Traffic Light Face: ('m/rO', '0E3j', [0., 0., 1.])


### Scene: ([  0, 248], 'host-a116', 1576254970817669888, 1576254995817669888)
  - [0, 248] : scene의 인덱스 범위 씬은 25초 길이의 연속된 데이터 조각을 의미
  - host-a116 : 데이터를 수집한 차량 id
  - 1576254970817669888, 1576254995817669888 씬의 시작과 끝을 나타내는 Unix 타임 스탬프(1970년 1월 1일 00:00:00 UTC부터의 초 단위 경과 시간을 나타내는 정수)

In [12]:
import datetime

timestamp = 1576254970817669888 / 1e9  # 나노초를 초로 변환
timestamp2 = 1576254995817669888 / 1e9
dt = datetime.datetime.utcfromtimestamp(timestamp)
dt2 = datetime.datetime.utcfromtimestamp(timestamp2)
print(dt.strftime('%Y-%m-%d %H:%M:%S'))
print(dt2.strftime('%Y-%m-%d %H:%M:%S'))

2019-12-13 16:36:10
2019-12-13 16:36:35


### Frame: (1576254971002364806, [ 0, 56], [0, 0], [  541.62249756, -2402.70581055,   288.62399292], [[-0.42925233, -0.90311611,  0.01112122], [ 0.90317595, -0.42916176,  0.00966374], [-0.00395468,  0.0141926 ,  0.99989146]])
  - 1576254971002364806 : 프레임의 타임스탬프
  - [0, 56]: 프레임 내의 객체 인덱스 범위로, 이 프레임에는 57개의 객체가 포함되어 있음을 나타냄
  - [0, 0] : 프레임 내의 라벨 인덱스 밤위(라벨이 존재하지 않음을 의미)
  - [  541.62249756, -2402.70581055,   288.62399292] : 3d 프레임 위치 좌표
  - [[-0.42925233, -0.90311611, 0.01112122], [0.90317595, -0.42916176, 0.00966374], [-0.00395468, 0.0141926, 0.99989146]]: 프레임 회전 행렬로 차량의 자세를 나타냄

### Agent: ([  545.47155762, -2411.68945312], [4.427541 , 1.8168873, 1.6035007], 2.0373902, [0., 0.], 1, [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
    - [545.47155762, -2411.68945312]: 에이전트의 2D 위치
    - [4.427541, 1.8168873, 1.6035007]: 에이전트의 크기를 나타내는 3D 치수
    - 2.0373902: 에이전트의 속도
    - [0., 0.]: 에이전트의 가속도
    - 1: 에이전트의 라벨(1은 보통 차량이라고 함)
    - [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]: 에이전트의 속도 및 가속도에 대한 추가 정보로, 각 요소는 특정 속성이나 상태를 나타냄

- [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.] 값 상세 속성
  - 속도_x: 에이전트의 x축 방향 속도 (m/s)
  - 속도_y: 에이전트의 y축 방향 속도 (m/s)
  - 속도_z: 에이전트의 z축 방향 속도 (m/s)
  - 가속도_x: 에이전트의 x축 방향 가속도 (m/s²)
  - 가속도_y: 에이전트의 y축 방향 가속도 (m/s²)
  - 가속도_z: 에이전트의 z축 방향 가속도 (m/s²)
  - 회전 속도_x: 에이전트의 x축 회전 속도 (rad/s)
  - 회전 속도_y: 에이전트의 y축 회전 속도 (rad/s)
  - 회전 속도_z: 에이전트의 z축 회전 속도 (rad/s)
  - 선형 속도: 에이전트의 선형 속도 크기 (m/s)
  - 선형 가속도: 에이전트의 선형 가속도 크기 (m/s²)
  - 선형 속도 표준 편차: 선형 속도의 표준 편차 (m/s)
  - 선형 가속도 표준 편차: 선형 가속도의 표준 편차 (m/s²)
  - 선형 속도 신뢰도: 선형 속도에 대한 신뢰도 점수 (0~1)
  - 선형 가속도 신뢰도: 선형 가속도에 대한 신뢰도 점수 (0~1)
  - 회전 속도 표준 편차: 회전 속도의 표준 편차 (rad/s)
  - 회전 속도 신뢰도: 회전 속도에 대한 신뢰도 점수 (0~1)

### Traffic Light Face: ('m/rO', '0E3j', [0., 0., 1.])
- 'm/rO(face_id) : 트래픽 라이트 고유 식별자
- '0E3j'(traffic_light_id) : 트래픽 라이트의 고유 식별자자
- [0., 0., 1.](traffic_light_face_status) : 차례대로 빨간불 노란불 초록불

## tfw 데이터 GeoTIFF(TIF)이미지 파일 위치 와 크기를 정의하는 파일 정보 읽기기
- 0.25              : 픽셀 크기 (X 방향)
- 0                 : 회전 값 (X 방향)
- 0                 : 회전 값 (Y 방향)
- -0.25             : 픽셀 크기 (Y 방향)
- -44628913.4205    : X 좌표
- 14701276.1451     : Y 좌표


- pb파일에 대해서 도 정리 하려했지만... 아직 열줄 모르겠네요..추가 할 수 있음 하겠습니다.